In [ ]:
#installing and importing pyspark

In [118]:
pip install pyspark


In [119]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [120]:
#creating the spark session
spark = SparkSession.builder.config("spark.driver.memory", "50g").appName("intern").getOrCreate()

In [121]:
spark

In [122]:
# importing the google drive for importing the datset because i am using google colab 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
#reading/creating the dataFrame with pyspark
df = spark.read.csv(
    "drive/My Drive/Colab Notebooks/internshala.csv", 
    header=True, 
    mode="DROPMALFORMED", 

)

In [126]:
### Check the schema
df.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- price_string: string (nullable = true)
 |-- price_string_unf: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- category: string (nullable = true)



In [127]:
type(df)

pyspark.sql.dataframe.DataFrame

In [128]:
df.show()

+--------------------+------------+----------------+--------------------+--------------------+--------------------+
|                uuid|price_string|price_string_unf|        product_type|             level_1|            category|
+--------------------+------------+----------------+--------------------+--------------------+--------------------+
|638744a4-b0ae-416...|        null|            null|TGFwdG9wIENvdmVyc...|     TGFwdG9wIENhc2U|     RWxlY3Ryb25pY3M|
|ab313969-02cc-48b...|        null|            null|QmFraW5nIEN1cHMgY...|QmFraW5nIE1hdHMgL...|a2l0Y2hpbmcgYW5kI...|
|acbd66ff-79f8-467...|      $19.95|            null|R3VtbWllcyB2aXRhb...|SW1tdW5pdHkgZ3Vtb...|            SGVhbHRo|
|963915d6-b2e3-409...|      $92.00|            null|            U2VydW1z|      RmFjZSBTZXJ1bQ|YmVhdXR5IGFuZCBwZ...|
|b5b68f3c-b1e0-40e...|       11.50|            null|RWF0aW5nIFV0ZW5za...|      Q2hvcHN0aWNrcw|a2l0Y2hpbmcgYW5kI...|
|389d9f75-cc3f-4bd...|        null|            null|TmF0dXJhbCBTd2Vld...

In [129]:
#filtering out all the null values from the "price_string" column
products_without_cost = df.filter(df.price_string.isNull())

In [130]:
products_without_cost.show()

+--------------------+------------+----------------+--------------------+--------------------+--------------------+
|                uuid|price_string|price_string_unf|        product_type|             level_1|            category|
+--------------------+------------+----------------+--------------------+--------------------+--------------------+
|638744a4-b0ae-416...|        null|            null|TGFwdG9wIENvdmVyc...|     TGFwdG9wIENhc2U|     RWxlY3Ryb25pY3M|
|ab313969-02cc-48b...|        null|            null|QmFraW5nIEN1cHMgY...|QmFraW5nIE1hdHMgL...|a2l0Y2hpbmcgYW5kI...|
|389d9f75-cc3f-4bd...|        null|            null|TmF0dXJhbCBTd2Vld...|      TW9uayBGcnVpdA|        Z3JvY2VyaWVz|
|1f2766ec-0a27-43f...|        null|            null|U2NydWJzIGFuZCBjb...|            U2NydWJz|SG91c2Vob2xkIGFuZ...|
|7bad39f5-74b9-461...|        null|            null|UGx1cyBzaXplIHdlY...|             RHJlc3M|Q2xvdGhpbmcgYW5kI...|
|524886b5-7cc2-4a5...|        null|            null| TWF0ZXJuaXR5IFBhZHM

In [131]:
#counting the products without the price/ where price_string == null
count_without_price = products_without_cost.groupBy('product_type','category','level_1').count()

In [132]:
count_without_price.show()

+--------------------+--------------------+--------------------+-----+
|        product_type|            category|             level_1|count|
+--------------------+--------------------+--------------------+-----+
|    Rm9vZCBTdG9yYWdl|a2l0Y2hpbmcgYW5kI...|V3JhcHMgYW5kIENvd...|20000|
|UGFwZXIgVG93ZWwgQ...|SG91c2Vob2xkIGFuZ...|        RGlzaGNsb3Ro|30000|
|TmF0dXJhbCBTd2Vld...|        Z3JvY2VyaWVz|            U3Rldmlh|10000|
|      QmFieSBXaXBlcw|         QmFieWNhcmU|      QmFieSBXaXBlcw|10000|
|TWF0ZXJuaXR5IFRza...|Q2xvdGhpbmcgYW5kI...|TWF0ZXJuaXR5IFNob...|30000|
|    RHV2ZXQgY292ZXJz|SG91c2Vob2xkIGFuZ...|c2F0ZWVuIGR1dmV0I...|20000|
|    TGVhc2hlcyBwZXRz|    cGV0IHN1cHBsaWVz|     RG9nIGxlYXNoZXM|20000|
|UGV0IEZsZWEgYW5kI...|    cGV0IHN1cHBsaWVz|RG9nL0NhdCBGbGVhI...|20000|
|        WW9nYSBNYXRz|            SGVhbHRo|RWNvIGZyaWVuZGx5I...|10000|
|cGV0IEJlZC9NYXR0c...|    cGV0IHN1cHBsaWVz|Y2F0IEJlZC9NYXR0c...|10000|
|        U2xlZXAgVCdz|Q2xvdGhpbmcgYW5kI...|  U2xlZXAgVHNoaXJ0cw|20000|
|U3R1Z

In [133]:
#droping the null values and assigning the dataframe in a new variable so our default dataframe stay untouched and orignal
without_null_price = df.na.drop(how="any",subset=['price_string'])

In [134]:
##counting the products with the price/ where price_string != null
count_with_price = without_null_price.groupBy('product_type','category','level_1').count()

In [135]:
count_with_price.show()

+--------------------+--------------------+--------------------+-----+
|        product_type|            category|             level_1|count|
+--------------------+--------------------+--------------------+-----+
|    Rm9vZCBTdG9yYWdl|a2l0Y2hpbmcgYW5kI...|V3JhcHMgYW5kIENvd...|10000|
|  Q3JhZnQgcmliYm9ucw|    QXJ0IHN1cHBsaWVz|  Q3JhZnQgcmliYm9ucw|10000|
|TmF0dXJhbCBTd2Vld...|        Z3JvY2VyaWVz|            U3Rldmlh|10000|
|V2F0ZXIgRmlsdGVyI...|VG9vbHMgYW5kIGhvb...|            MzAgQ3Vw|10000|
|  U2hhdmluZyBDcmVhbQ|YmVhdXR5IGFuZCBwZ...|  U2hhdmluZyBDcmVhbQ|10000|
|      QmFieSBXaXBlcw|         QmFieWNhcmU|      QmFieSBXaXBlcw|10000|
|    RHV2ZXQgY292ZXJz|SG91c2Vob2xkIGFuZ...|c2F0ZWVuIGR1dmV0I...|10000|
|UGV0IEZsZWEgYW5kI...|    cGV0IHN1cHBsaWVz|RG9nL0NhdCBGbGVhI...|10000|
|        WW9nYSBNYXRz|            SGVhbHRo|RWNvIGZyaWVuZGx5I...|20000|
|    TGVhc2hlcyBwZXRz|    cGV0IHN1cHBsaWVz|     RG9nIGxlYXNoZXM|10000|
|    RmFjZSBtYWtlIHVw|YmVhdXR5IGFuZCBwZ...|          QnJvbnplcg|20000|
|     

In [136]:
#importing all functions form pyspark.sql.functions
from pyspark.sql.functions import *

import pyspark.sql.functions as F

In [137]:
#removing all "$" signs from the price_string for making this column to values
currency = without_null_price.withColumn('price_string', F.regexp_replace('price_string', r'^[$]*', ''))


In [138]:
currency.show()

+--------------------+------------+----------------+--------------------+--------------------+--------------------+
|                uuid|price_string|price_string_unf|        product_type|             level_1|            category|
+--------------------+------------+----------------+--------------------+--------------------+--------------------+
|acbd66ff-79f8-467...|       19.95|            null|R3VtbWllcyB2aXRhb...|SW1tdW5pdHkgZ3Vtb...|            SGVhbHRo|
|963915d6-b2e3-409...|       92.00|            null|            U2VydW1z|      RmFjZSBTZXJ1bQ|YmVhdXR5IGFuZCBwZ...|
|b5b68f3c-b1e0-40e...|       11.50|            null|RWF0aW5nIFV0ZW5za...|      Q2hvcHN0aWNrcw|a2l0Y2hpbmcgYW5kI...|
|9599f1a9-d406-43e...|       24.99|            null|TW9wcyBhbmQgYnJvb21z|                TW9w|SG91c2Vob2xkIGFuZ...|
|35799087-f6f4-4ca...|      148.00|            null|    V29tZW5zIFBhbnRz|V29tZW5zIFJlZ3VsY...|Q2xvdGhpbmcgYW5kI...|
|9b3f553e-ee4c-4e1...|          89|            null|    V29tZW5zIFBhbnRz

In [139]:
#creating the new column currency with the currency as the values
currency = without_null_price.withColumn("currency", lit('US DOLLAR'))

In [140]:
currency.show()

+--------------------+------------+----------------+--------------------+--------------------+--------------------+---------+
|                uuid|price_string|price_string_unf|        product_type|             level_1|            category| currency|
+--------------------+------------+----------------+--------------------+--------------------+--------------------+---------+
|acbd66ff-79f8-467...|      $19.95|            null|R3VtbWllcyB2aXRhb...|SW1tdW5pdHkgZ3Vtb...|            SGVhbHRo|US DOLLAR|
|963915d6-b2e3-409...|      $92.00|            null|            U2VydW1z|      RmFjZSBTZXJ1bQ|YmVhdXR5IGFuZCBwZ...|US DOLLAR|
|b5b68f3c-b1e0-40e...|       11.50|            null|RWF0aW5nIFV0ZW5za...|      Q2hvcHN0aWNrcw|a2l0Y2hpbmcgYW5kI...|US DOLLAR|
|9599f1a9-d406-43e...|      $24.99|            null|TW9wcyBhbmQgYnJvb21z|                TW9w|SG91c2Vob2xkIGFuZ...|US DOLLAR|
|35799087-f6f4-4ca...|     $148.00|            null|    V29tZW5zIFBhbnRz|V29tZW5zIFJlZ3VsY...|Q2xvdGhpbmcgYW5kI...|US 

In [141]:
#changing the name of the price_string to value
currency=currency.withColumnRenamed('price_string', 'value')


In [142]:
currency.show()

+--------------------+-------+----------------+--------------------+--------------------+--------------------+---------+
|                uuid|  value|price_string_unf|        product_type|             level_1|            category| currency|
+--------------------+-------+----------------+--------------------+--------------------+--------------------+---------+
|acbd66ff-79f8-467...| $19.95|            null|R3VtbWllcyB2aXRhb...|SW1tdW5pdHkgZ3Vtb...|            SGVhbHRo|US DOLLAR|
|963915d6-b2e3-409...| $92.00|            null|            U2VydW1z|      RmFjZSBTZXJ1bQ|YmVhdXR5IGFuZCBwZ...|US DOLLAR|
|b5b68f3c-b1e0-40e...|  11.50|            null|RWF0aW5nIFV0ZW5za...|      Q2hvcHN0aWNrcw|a2l0Y2hpbmcgYW5kI...|US DOLLAR|
|9599f1a9-d406-43e...| $24.99|            null|TW9wcyBhbmQgYnJvb21z|                TW9w|SG91c2Vob2xkIGFuZ...|US DOLLAR|
|35799087-f6f4-4ca...|$148.00|            null|    V29tZW5zIFBhbnRz|V29tZW5zIFJlZ3VsY...|Q2xvdGhpbmcgYW5kI...|US DOLLAR|
|9b3f553e-ee4c-4e1...|    $89|  

In [143]:
#Listing out the categories with average price of product.
avg_p_category = currency.groupBy('category').agg(F.mean("value"))

In [144]:
avg_p_category.show()

+--------------------+------------------+
|            category|        avg(value)|
+--------------------+------------------+
|         QmFieWNhcmU|              null|
|    QXJ0IHN1cHBsaWVz|              null|
|    cGV0IHN1cHBsaWVz|              null|
|                null|              null|
|SG91c2Vob2xkIGFuZ...|             108.0|
|b2ZmaWNlIHByb2R1Y3Rz|              null|
|a2l0Y2hpbmcgYW5kI...|18.189333333333394|
|            SGVhbHRo|              null|
|Q2xvdGhpbmcgYW5kI...|              52.0|
|     RWxlY3Ryb25pY3M|              null|
|        Z3JvY2VyaWVz|              null|
|YmVhdXR5IGFuZCBwZ...|              null|
|VG9vbHMgYW5kIGhvb...|              null|
| VG95cyBhbmQgR2FtZXM|14.949999999999168|
+--------------------+------------------+



In [145]:
pip freeze > requirements.txt